<a href="https://colab.research.google.com/github/fabishevskiy/traffic_analysis/blob/main/%D0%9F%D0%BE%D0%B4%D0%B1%D0%BE%D1%80_%D0%BB%D0%B8%D1%84%D1%82%D0%BE%D0%B2_%D1%80%D0%B0%D1%81%D1%87%D0%B5%D1%82%D0%BD%D1%8B%D0%BC_%D0%BC%D0%B5%D1%82%D0%BE%D0%B4%D0%BE%D0%BC_%D0%93%D0%9E%D0%A1%D0%A2_34758_2021%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Описание проекта

Настоящий анализ трафика основан на расчетном методе в соответствие с *ГОСТ 34758-2021 ЛИФТЫ. Определение числа, параметров и размеров лифтов для зданий различного назначения*.  
Данный метод основан на условиях пикового пассажиропотока вверх (входящий пассажиропоток). При этом основной посадочный этаж находится внизу здания, все верхние этажи заселены равномерно (п.5.4.1, ГОСТ 34758-2021).


Рекомендуемые критерии для расчетного метода анализа по ГОСТ 34758-2021:  
Таблица 1



Назначение здания  | Расчетный пассажиропоток, % |Нормативный интервал движения, с
-------------------|-----------------------------|-----------------------------
Офис               | ≥ 12                        | ≤30
Гостиница          | ≥ 12                        | ≤40
Жилое здание       | ≥6                          | ≤60


Таблица 4 - Нормативное время движения лифта

Назначение здания  | Нормативное время движения лифта, с
-------------------|------------------------------------
Офис               | 20-30
Гостиница          | 25-35
Жилое здание       | 25-45


In [ ]:
# @title # Исходные данные по зданию

# импорт библиотек

from IPython.display import Markdown as md
from IPython.display import Latex as lt
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# зададим константные значения
T_5 = 5*60             # время для расчета 5 минутного интервала = 300 сек
a_n = 1.0            # ускорение/замедление лифта
j_n = 1.0             # рывок
PASSENGER_WEIGHT = 75  # вес одного пассажира
F_1 = 0.8              # коэффициент загрузки кабины

# список типов здания
building_types = ['Офис', 'Гостиница', 'Жилое здание']

# список нормальных скоростей лифта
v_nom_list = [1.0, 1.6, 1.75, 2.0, 2.5, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 100.0]

# нормативный интервал движения лифтов для различных типов здания, t_и
t_int_req = {
    'Офис': 30,
    'Гостиница': 40,
    'Жилое здание': 60,
}

# Нормативный пассажиропоток для различных типов здания
C_h_req = {
    'Офис': 12,
    'Гостиница': 12,
    'Жилое здание': 6
}

# минимальное рекомендуемое время движения лифта на всю высоты подъема
t_H_min_req = {
    'Офис': 20,
    'Гостиница': 25,
    'Жилое здание': 25
}

# максимальное рекомендуемое время движения лифта на всю высоты подъема
t_H_max_req ={
    'Офис': 30,
    'Гостиница': 35,
    'Жилое здание': 45
}

# Выберем тип здания
# 0 - Офис
# 1 - Гостиница
# 2 - Жилое здание
building_type = building_types[0] # <------- ввести тип здания

'''
# Ввести исходные значения по зданию с клавиатуры
building_type = building_types[int(input('Укажите тип здания (0 - офис, 1 - гостиница, 2 - Жилое здание): '))]    # Выберем тип здания
U = int(input('Общая заселенность здания (человек): '))           # общая заселенность здания
D = int(input('Высота подъема лифта (м): '))                      # высота подъема лифта
d_f = int(input('Расстояние между этажами (м): '))                # расстояние между этажами, м;
N = int(input('Количество обслуживаемых этажей выше основного посадочного: '))   # количество обслуживаемых этажей выше основного посадочного

'''

# общая заселенность здания

A = 1362               # общая заселенность здания
h_f = 4.05             # расстояние между этажами, м; h_э - добавить формулу расчета высоты подъема
N = 9                # количество обслуживаемых этажей выше основного посадочного N_эт
H_max = h_f * N       # высота подъема лифта

# расчитаем интервал движения лифтов и пассажиропоток для выбранного типа здания
t_int = t_int_req[building_type]       # требуемый интервал движения
A_capacity = C_h_req[building_type]    # пассажиропоток
t_H_min = t_H_min_req[building_type]   # мин время поддъема
t_H_max = t_H_max_req[building_type]   # макс время поддъема

# print(f'Характеристики по зданию: \nОбщая заселенность здания A = {A} человек, \nВысота подъема лифта D = {D} метров \nРасстояние между этажами d = {d_f} метров, \nКоличество обслуживаемых этажей выше основной этажной площадки N = {N} этажей\n')
display(md(f'Характеристики по зданию:'))
display(md(f'$A$ = {A} человек - общая заселенность здания;'))
display(md(f'$H_{{max}}$= {H_max:.2f} м - высота подъема лифта;'))
display(md(f'$h_{{э}}$= {h_f} м - высота между этажами;'))
display(md(f'$N_{{эт}}$= {N} количество обслуживаемых этажей (выше основной посадочной площадки).'))
display(md(f'Тип здания - {building_type}.  \n  \nДля данного типа зданий предъявляются следующие критерии подбора лифтового оборудования (провозные характеристики): '))
display(md(f'- провозная способность лифтов $\%P_{{5}}$: не менее {A_capacity}% населения здания за 5 мин;'))
display(md(f'- интервал движения лифтов $t_{{и}}$ при нормативном пассажиропотоке: не более {t_int} сек;'))
display(md(f'- время подъема на всю высоту подъема $t_{{H}}$: от {t_H_min} до {t_H_max} сек.'))

Характеристики по зданию:

$A$ = 1362 человек - общая заселенность здания;

$H_{max}$= 36.45 м - высота подъема лифта;

$h_{э}$= 4.05 м - высота между этажами;

$N_{эт}$= 9 количество обслуживаемых этажей (выше основной посадочной площадки).

Тип здания - Офис.  
  
Для данного типа зданий предъявляются следующие критерии подбора лифтового оборудования (провозные характеристики): 

- провозная способность лифтов $\%P_{5}$: не менее 12% населения здания за 5 мин;

- интервал движения лифтов $t_{и}$ при нормативном пассажиропотоке: не более 30 сек;

- время подъема на всю высоту подъема $t_{H}$: от 20 до 30 сек.

In [ ]:
# @title #Расчеты

# Количество людей, которых необходимо перевезти за 5 мин - необходимая провозная способность
A_5 = A * A_capacity/100
display(md(f'При общей заселенности здания {A} чел., за 5 минут необходимо перевезти:  \n $A_{{5}} = A \cdot \%P_{{5}} / 100 = {A} \cdot {{{A_capacity}}} / 100 = {A_5:.2f} $ чел. - расчетный пассажиропоток в течение 5 мин.'))

# Количество отправлений лифтов с основного посадочного этажа за 5 минут
n_rt = math.ceil(T_5 / t_int)
display(md(f'Минимальное количество отправлений лифтов с основного посадочного этажа для обеспечения требуемого интервала {t_int} с:  \n$n_{{отпр}} =5 \cdot 60 / t_{{и}} = 300 /{t_int} = {n_rt}$.'))

# Высчислим среднюю загрузку кабины, для чего разделим количество человек, перевозимых за 5 минут, на количество рейсов.
P_k = A_5 / n_rt # Расчетная средняя загрузка кабины
display(md(f' Средняя загрузка кабины лифта:  \n $P_{{к}} = A_{{5}} / n_{{отпр}} = {A_5} / {n_rt} = {P_k:.2f}$ чел.'))

# Скорость лифтов
v_n_max = H_max / t_H_min
v_n_min = H_max / t_H_max
display(md(f'Скорость лифтов должна быть в следующих пределах:  \n от $H_{{max}}/{t_H_max}={H_max:.2f}/{t_H_max}={v_n_min:.1f}$ м/с   \n до $H_{{max}}/{t_H_min}={H_max:.2f}/{t_H_min}={v_n_max:.1f}$ м/с'))

def v_nom_selection(v_min, v_nom_list):
    for v in v_nom_list:
        if v > v_min:
            return v

v_n = v_nom_selection(v_n_min, v_nom_list)
#v_n = 2.5
display(md(f'Примем номинальную скорость лифтов равной:  \n$v_{{n}}={v_n}$ с'))

# - время подъема лифта на всю высоту без учета ускорения, рывка
t_H_nom = H_max / v_n
# - время подъема лифта на всю высоту с учетом ускорения и рывка
t_H_real = H_max / v_n + v_n/a_n + a_n/j_n
display(md(f'Время подъема лифта на всю высоту:  \n - на номинальной скорости без учета ускорения и рывка.'))

display(md(rf'$t_{{Hn}}=H_{{max}} / v_{{n}}={H_max:.2f}/{v_n}={t_H_nom:.2f}$ с'))
display(md(rf'- с учетом ускорения и рывка:'))
display(lt(rf'$t_{{H}}=\frac{{H_{{max}}}}{{v_{{n}}}}+\frac{{v_{{n}}}}{{a_{{n}}}}+\frac{{a_{{n}}}}{{j_{{n}}}}=\frac{{{H_max:.2f}}}{{{v_n}}}+\frac{{{v_n}}}{{{a_n}}} +\frac{{{a_n}}}{{{j_n}}}={t_H_real:.2f}\,с$'))

# Время, затрачиваемое на остановку
display(md(f'Рассчитаем время, затрачиваемое на остановку по формуле (9) ГОСТ 34758-2021:'))
display(md(f'$t_{{ост}} = t_{{з}} + t_{{з.д}} + t_{{эт}} - t_{{пр}} + t_{{о}} - t_{{з.з}}- t_{{эт.н}}$'))

# время движения лифта между соседними этажами с учетом стадии разгона и торможения;
def t_f1_calc(h_floor, v_n, a_n, j_n): #v_n - speed, a_n - accelertion, j_n - jerk
    # Расстояние, пройденное лифтом от начала движения до достижения номинальной скорости:
    s_vm = v_n ** 2 / (2 * a_n) + (a_n * v_n)/(2 * j_n)
    if s_vm > h_floor / 2:
        v_m = - a_n ** 2 / (2 * j_n) + ((a_n * h_floor) + (a_n ** 2 / j_n) ** 2) ** (0.5)
        return h_floor / v_m + a_n / j_n + v_m / a_n
        return a_n / j_n + (4 * h_floor / a_n + (a_n / (2 * j_n)) ** 2) ** (0.5)
    return h_floor / v_n + a_n / j_n + v_n / a_n


t_sd = 0.6                             # время задержки начала движения, с; / start delay time
t_f1 = t_f1_calc(h_f, v_n, a_n, j_n)   # время движения лифта между соседними этажами с учетом стадии разгона и торможения; / flight time
t_pre = 0.5                            # время предварительного открывания двери, с; / pre-opening time
t_cd = 2.0                             # время задержки закрывания двери, с; / closing delay time
t_v = h_f / v_n                        # время движения лифта между соседними этажами на номинальной скорости, с; / travel berween two floor at rated speed (s)


# время открывания / закрывания-открывания двери / входа-выхода пассажиров
# ширина дверного проема: время входа-выхода, центральные-открывание, центральные-закрывание, телескопические-открывание, телескопические-закрывание
door_list = {
    800:  [1.2, 1.4, 1.8, 2.5, 2.9],
    900:  [1.1, 1.6, 2.0, 2.6, 3.0],
    1000: [1.0, 1.8, 2.2, 2.8, 3.3],
    1200: [0.9, 2.2, 2.6, 3.4, 3.9]
}
door_tipes = ('центрального', 'телескопического')

door_width = 900     # 800, 900, 1000, 1200
door_type = 1        # 0-центральные, 1-телескопические

t_c = door_list[door_width][2+2*door_type] # время закрывания двери, с;  / closing time
t_o = door_list[door_width][1+2*door_type] # время открывания двери, с; / opening time
t_p = door_list[door_width][0]             # время на вход пассажира в лифт, с # время на вход пассажира в лифт - из табл.3 ГОСТ 34758-2021

display(md(f'Для дверей {door_tipes[door_type]} открывания шириной проема {door_width} мм:'))
display(md(f'$t_{{з}}={t_c}$ с - время закрывания двери;'))
display(md(f'$t_{{о}}={t_o}$ с - время открывания двери;'))
display(md(f'$t_{{з.д}}={t_sd}$ с - время задержки начала движения;'))
display(md(f'$t_{{эт}}={t_f1:.2f}$ с - время движения лифта между соседними этажами с учетом стадии разгона и торможения;'))
display(md(f'$t_{{пр}}={t_pre}$ с - время предварительного открывания двери;'))
display(md(f'$t_{{з.з}}={t_cd}$ с - время задержки закрывания двери;'))
display(md(f'$t_{{эт.н}}={t_v:.2f}$ с - время движения между соседними этажами на номинальной скорости.'))
t_s = t_c + t_sd + t_f1 -t_pre + t_o + t_cd - t_v # время затрачиваемое на остановку

display(md(f'получаем:  \n$t_{{ост}} = t_{{з}} + t_{{з.д}} + t_{{эт}} - t_{{пр}} + t_{{о}} - t_{{з.з}}- t_{{эт.н}}= {t_c} + {t_sd} + {t_f1:.2f} - {t_pre} + {t_o} - {t_v:.2f} = {t_s:.2f} с$'))

# Функция для расчета этажа реверса
def av_highest_reversal_floor(N, P_k):
    sum = 0
    sum_i = [] # запишем значения для вывода в формулу на экран
    for i in range(1, N):
        sum += (i/N)**P_k
        sum_i.append(sum)
    return N - sum #, sum_i # Номер этажа реверса при круговом рейсе

# Функция формулы для расчета этажа реверса
def formula_highest_reversal_floor(N, P_k):
    sum = 0
    sum_i = []
    text_formulas = ''
    text_result=''

    for i in sum_i:
        if i != 0:
            text_formulas += ' + '
            text_results  += ' + '
        text_i += f'{i:.8f}'
    return text_formulas, text_results

# определим этажа реверса
h_rev = av_highest_reversal_floor(N, P_k)

print(f'Средний этаж реверса:')
display(lt(rf'$ H = N_{{эт}} - \sum_{{i=1}}^{{N_{{эт}}-1}}(\frac{{i}}{{N_{{эт}}}})^{{P_{{к}}}} = {{{N}}} - \sum_{{i=1}}^{{{{{N}}}-1}}(\frac{{i}}{{{N}}})^{{{P_k:.2f}}} = {h_rev:.2f} $'))

# Вероятное число остановок
display(md(rf'Определим вероятное число остановок при равномерной заселенности этажей :'))
S = N * (1 - (1 - 1 / N)**P_k)
display(lt(rf'$ S = N_{{эт}}[1-(1-\frac{{1}}{{N_{{эт}}}})^{{P_{{к}}}}]= {{{N}}}[1-(1-\frac{{1}}{{{N}}})^{{{P_k:.2f}}}]={{{S:.2f}}}$'))

print('\n\n\n')

При общей заселенности здания 1362 чел., за 5 минут необходимо перевезти:  
 $A_{5} = A \cdot \%P_{5} / 100 = 1362 \cdot {12} / 100 = 163.44 $ чел. - расчетный пассажиропоток в течение 5 мин.

Минимальное количество отправлений лифтов с основного посадочного этажа для обеспечения требуемого интервала 30 с:  
$n_{отпр} =5 \cdot 60 / t_{и} = 300 /30 = 10$.

 Средняя загрузка кабины лифта:  
 $P_{к} = A_{5} / n_{отпр} = 163.44 / 10 = 16.34$ чел.

Скорость лифтов должна быть в следующих пределах:  
 от $H_{max}/30=36.45/30=1.2$ м/с   
 до $H_{max}/20=36.45/20=1.8$ м/с

Примем номинальную скорость лифтов равной:  
$v_{n}=1.6$ с

...

получаем:  
$t_{ост} = t_{з} + t_{з.д} + t_{эт} - t_{пр} + t_{о} - t_{з.з}- t_{эт.н}= 3.0 + 0.6 + 5.07 - 0.5 + 2.6 - 2.53 = 10.23 с$

Средний этаж реверса:


<IPython.core.display.Latex object>

Определим вероятное число остановок при равномерной заселенности этажей :

<IPython.core.display.Latex object>

In [ ]:
# @title Расчеты (продолжение)


# время кругового рейса
t_rt = 2*h_rev*t_v + (S+1)*t_s + 2*P_k*t_p
display(md(rf'Время кругового рейса:'))
display(lt(rf't_{{rt}}=2\cdot H_{{max}}\cdot t_{{v}}+ (S+1)\cdot t_{{s}}+2\cdot P_{{к}}\cdot t_{{p}}=2\cdot {{{h_rev:.2f}}}\cdot {{{t_v:.2f}}}+ ({{{S:.2f}}}+1)\cdot {{{t_s:.2f}}}+2\cdot {{{P_k:.2f}}}\cdot {{{t_p:.2f}}} = {{{t_rt:.2f}}}\, с'))


#  количество лифтов
N_l = math.ceil(t_rt / t_int)
# N_l = 4
display(md(rf'Минимальное требуемое количество лифтов определяется делением времени кругового рейса на требуемый интервал с округлением до целого вверх:'))
display(lt(rf't_{{rt}}/t_{{и}}={{{t_rt:.2f}}}/{{{t_int}}}={{{t_rt/t_int:.2f}}}'))
display(lt(rf'N_{{л}}={{{N_l}}}\, лифта(ов)'))

# Расчет минимальной грузоподъемности одного лифта
Q_min = P_k * PASSENGER_WEIGHT / F_1
display(md(rf'С учетом коэффициента загрузки кабины $F_{{1}} = {{{F_1}}}$, рассчитаем минимальную требуемую грузоподъемность лифтов:'))
display(lt(rf'Q_{{мин}}=(P_{{к}}\cdot 75 кг) / F_{{1}}=({{{P_k:.2f}}}\cdot {{{PASSENGER_WEIGHT}}})/{{{F_1}}}={{{Q_min:.2f}}}'))
Q_list=[400, 450, 630, 825, 1000, 1275, 1600, 2000, 5000] # Список номинальных грузоподъемностей
Q_n = min([q for q in Q_list if q > Q_min])               # Расчет номинальной грузоподъемности лифтов
display(md(f'Примем номинальную грузоподъемность лифтов:  \n $Q_{{n}} = {{{Q_n}}}$ кг'))
print()
print()

Время кругового рейса:

<IPython.core.display.Latex object>

Минимальное требуемое количество лифтов определяется делением времени кругового рейса на требуемый интервал с округлением до целого вверх:

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

С учетом коэффициента загрузки кабины $F_{1} = {0.8}$, рассчитаем минимальную требуемую грузоподъемность лифтов:

<IPython.core.display.Latex object>

Примем номинальную грузоподъемность лифтов:  
 $Q_{n} = {1600}$ кг

In [ ]:
# @title Проверка провозных характеристик выбранных лифтов
display(md(rf'Уточним провозные характеристики лифтовой группы из {N_l} лифтов номинальной скоростью {v_n} м/с и г/п {Q_n} кг, для средней загрузки $P_{{к}}={{{P_k:.2f}}}$ чел.:'))

t_int_n = t_rt / N_l
display(md(f'Интервал движения {N_l} лифтов:  \n $t_{{и}}=t_{{rt}}/ N_{{л}} = {{{t_rt:.2f}}} / {{{N_l}}} = {{{t_int_n:.2f}}}$ с'))

C_h = (T_5 * P_k) / t_int_n
display(md(f'Число пассажиров, перевозимых за 5 мин (300 сек):  \n $С_{{h}} = 300\cdot P_{{к}} / t_{{и}}=300\cdot {{{P_k:.2f}}} / {{{t_int_n:.2f}}} = {{{C_h:.2f}}} $ чел.'))

C_h_p = C_h / A * 100
display(md(f'Процент населения здания, перевозимого за 5 мин:  \n $\% C_{{h}} =С_{{h}}/A\cdot 100\% = {{{C_h:.2f}}}/{{{A}}}\cdot 100\% = {{{C_h_p:.2f}}}\%$'))

F_k = P_k * PASSENGER_WEIGHT / Q_n * 100
display(md(f'При этом коэффициент загрузки кабин равен: \n$ F_{{к}} = (P_{{к}}\cdot 75) / Q_{{n}} = ({{P_k}}\cdot 75) / {{Q_n}} = {{{F_k:.0f}}} \%$'))

Уточним провозные характеристики лифтовой группы из 6 лифтов номинальной скоростью 1.6 м/с и г/п 1600 кг, для средней загрузки $P_{к}={16.34}$ чел.:

Интервал движения 6 лифтов:  
 $t_{и}=t_{rt}/ N_{л} = {157.04} / {6} = {26.17}$ с

Число пассажиров, перевозимых за 5 мин (300 сек):  
 $С_{h} = 300\cdot P_{к} / t_{и}=300\cdot {16.34} / {26.17} = {187.34} $ чел.

Процент населения здания, перевозимого за 5 мин:  
 $\% C_{h} =С_{h}/A\cdot 100\% = {187.34}/{1362}\cdot 100\% = {13.75}\%$

При этом коэффициент загрузки кабин равен: 
$ F_{к} = (P_{к}\cdot 75) / Q_{n} = ({P_k}\cdot 75) / {Q_n} = {77} \%$

In [ ]:
# @title #Вывод
display(md(f'Лифтовая группа из  {N_l} лифтов  номинальной скоростью {v_n} м/с и г/п {Q_n} кг обеспечивает соблюдение предъявляемых критериев провозной способности.'))

Лифтовая группа из  6 лифтов  номинальной скоростью 1.6 м/с и г/п 1600 кг обеспечивает соблюдение предъявляемых критериев провозной способности.